In [ ]:
!python -m pip install poliastro
!python -m pip install sbpy
!python -m pip install spiceypy
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls
import numpy as np
import spiceypy as spy
spy.furnsh("naif0012.tls") #Cargar Kernel de datos 
import numpy as np 
import datetime
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
from sbpy.data import Ephem
from astropy.time import Time
from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody.orbit import Orbit
from poliastro.plotting import OrbitPlotter3D


deg=np.pi/180 # para pasar los alfa y delta a radianes
rad=1/deg
au=149597870.693 # km, fuente: https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/de-403-masses.tpc
mu=132712440023.31

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 143 kB 8.6 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 4.5 MB 45.0 MB/s 
     |████████████████████████████████| 832 kB 33.9 MB/s 
     |████████████████████████████████| 48 kB 6.0 MB/s 
     |████████████████████████████████| 4.1 MB 40.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 9.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 914 kB 43.9 MB/s 
     |████████████████████████████████| 41 kB 664 kB/s 
  Created wheel for sbpy: filename=sbpy-0.3.0-py3-none-any.whl size=447692 sha256=f3c897fb4be9c9da42a3c6a777ebed49bff8fb0ca5e1c251a966ab0296b5f2c5
  Stored in dir

# DATOS OBSERVACIONALES

In [ ]:
def posicion_celeste(a,d):
  alfa=15*(a[0]+a[1]/60+a[2]/3600)
  delta=d[0]+d[1]/60+d[2]/3600
  return alfa,delta

In [ ]:
#Primera observación
alfa1,delta1=posicion_celeste([14,20,8.22],[-22,24,41.8]) 

#Segunda observación
alfa2,delta2=posicion_celeste([14,49,25.66],[ -21,57,55.9])   
    
#Tercera observación
alfa3,delta3=posicion_celeste([15,31,34.64],[-22,42,22.5])  

alfa1,alfa2,alfa3,delta1,delta2,delta3

(215.03425,
 222.35691666666665,
 232.89433333333335,
 -21.58838888888889,
 -20.034472222222224,
 -21.29375)

# TIEMPO DE EFEMERIDES

In [ ]:
def tiempo_astronomico(fecha):
  tu=spy.str2et(fecha) 
  dt=spy.deltet(tu,"ET") 
  t=tu-dt
  return (t)

In [ ]:
fecha1="2020-07-15"
fecha2="2020-08-15" 
fecha3="2020-09-15"

In [ ]:
t1=tiempo_astronomico(fecha1)
t2=tiempo_astronomico(fecha2)
t3=tiempo_astronomico(fecha3)
t1,t2,t3 #Esta en segundos

(648043200.0, 650721600.0, 653400000.0)

# POSICIÓN TIERRA-SOL

In [ ]:
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon
import astropy.units as u
!pip install jplephem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def posicion_tierra(Ra):
  t = Time(Ra)
  P=get_body_barycentric('earth',t,ephemeris='de432s')
  
  return P

In [ ]:
#Posición para fecha uno de observación
R1=posicion_tierra(fecha1)

#Posición para fecha dos de observación
R2=posicion_tierra(fecha2)

#Posición para fecha tres de observación
R3=posicion_tierra(fecha3)

R1,R2,R3

(<CartesianRepresentation (x, y, z) in km
     (57870993.82161374, -1.27776518e+08, -55379504.72733801)>,
 <CartesianRepresentation (x, y, z) in km
     (1.1912605e+08, -83993139.82766682, -36398905.35409363)>,
 <CartesianRepresentation (x, y, z) in km
     (1.48223919e+08, -17516522.46122035, -7580744.11079644)>)

In [ ]:
'''Ya que la representación cartesiana no permite manipular los datos
se extraen las cantidades omitiendo las unidades. Se usa el signo
 menos para referir los vectores desde el Sol.'''
P1=[-R1.x/u.km,-R1.y/u.km,-R1.z/u.km]
P2=[-R2.x/u.km,-R2.y/u.km,-R2.z/u.km]
P3=[-R3.x/u.km,-R3.y/u.km,-R3.z/u.km]
P1,P2,P3

([<Quantity -57870993.82161374>,
  <Quantity 1.27776518e+08>,
  <Quantity 55379504.72733801>],
 [<Quantity -1.1912605e+08>,
  <Quantity 83993139.82766682>,
  <Quantity 36398905.35409363>],
 [<Quantity -1.48223919e+08>,
  <Quantity 17516522.46122035>,
  <Quantity 7580744.11079644>])

# VECTORES UNITARIOS DE OBSERVACIÓN

In [ ]:
#Vector unitario para la primera observación
u1=np.array([np.cos(delta1*deg)*np.cos(alfa1*deg),np.cos(delta1*deg)*np.sin(alfa1*deg),np.sin(delta1*deg)])

#Vector unitario para la segunda observación
u2=np.array([np.cos(delta2*deg)*np.cos(alfa2*deg),np.cos(delta2*deg)*np.sin(alfa2*deg),np.sin(delta2*deg)])

#Vector unitario para la tercera observación
u3=np.array([np.cos(delta3*deg)*np.cos(alfa3*deg),np.cos(delta3*deg)*np.sin(alfa3*deg),np.sin(delta3*deg)])

u1,u2,u3 

(array([-0.76137045, -0.53379589, -0.36793612]),
 array([-0.69424511, -0.63297625, -0.34258545]),
 array([-0.56210098, -0.74307796, -0.3631496 ]))

# COEFICIENTES DE LAGRAGE

In [ ]:
r2=3*au #Valor arbitrario

f3=1-mu/(2*r2**3)*(t3-t2)**2
g3=(t3-t2)-mu/(6*r2**3)*(t3-t2)**3

f1=1-mu/(2*r2**3)*(t1-t2)**2
g1=(t1-t2)-mu/(6*r2**3)*(t1-t2)**3

# CONSTANTES

In [ ]:
c1=g3/(f1*g3-f3*g1)
c3=-g1/(f1*g3-f3*g1)

c1,c3

(0.5026470101979923, 0.5026470101979923)

# VALOR DE **pho2**

In [ ]:
rho2=(np.dot(P2,np.cross(u1,u3))-c1*np.dot(P1,np.cross(u1,u3))-c3*np.dot(P3,np.cross(u1,u3)))/np.dot(u2,np.cross(u1,u3))

rho2/au

1.3960749517720832

# VALOR DE **R2**

In [ ]:
r2=np.sqrt(np.linalg.norm(P2)**2+rho2**2-2*rho2*np.dot(P2,u2))

r2/au

1.6245059897827647

## CICLO PARA ENCONTRAR EL VALOR VERDADERO DE **R2**

In [ ]:
r2=3*au
for i in range(30):
  f3=1-mu/(2*r2**3)*(t3-t2)**2
  g3=(t3-t2)-mu/(6*r2**3)*(t3-t2)**3

  f1=1-mu/(2*r2**3)*(t1-t2)**2
  g1=(t1-t2)-mu/(6*r2**3)*(t1-t2)**3

  c1=g3/(f1*g3-f3*g1)
  c3=-g1/(f1*g3-f3*g1)

  rho2=(np.dot(P2,np.cross(u1,u3))-c1*np.dot(P1,np.cross(u1,u3))-c3*np.dot(P3,np.cross(u1,u3)))/np.dot(u2,np.cross(u1,u3))

  r2=(np.linalg.norm(P2)**2+rho2**2-2*rho2*np.dot(P2,u2))**0.5
  
  print(r2/au)
  

1.6245059897827647
1.422198777699357
1.3105309320635017
1.2227674008091312
1.1395087147303489
1.0563640076077445
0.9996481692973289
1.0186319151994705
1.0035623034772423
1.0146412946652157
1.0058715869770651
1.0125127001238903
1.0072808541306106
1.0112937069726846
1.0081451671363615
1.0105755715784532
1.0086741891009916
1.0101469608339286
1.0089970036173979
1.009889472574634
1.0091934912813234
1.0097342453839946
1.0093128743319915
1.0096404839577504
1.0093853245531412
1.0095837860537669
1.009429259738513
1.0095494780902725
1.0094558904753768
1.0095287102122426


# VALOR DE **pho1** Y **pho3**

In [ ]:
rho1=(-np.dot(P2,np.cross(u2,u3))+c1*np.dot(P1,np.cross(u2,u3))+c3*np.dot(P3,np.cross(u2,u3)))/(c1*np.dot(u1,np.cross(u2,u3)))
rho3=(-np.dot(P2,np.cross(u2,u1))+c1*np.dot(P1,np.cross(u2,u1))+c3*np.dot(P3,np.cross(u2,u1)))/(c3*np.dot(u3,np.cross(u2,u1)))

rho1/au,rho3/au

(-0.019163447273443202, -0.04911401392501752)

# VALOR DE **r1** Y **r3**

In [ ]:
r1=np.sqrt(np.linalg.norm(P1)**2+rho1**2-2*rho1*np.dot(P1,u1))
r3=np.sqrt(np.linalg.norm(P3)**2+rho3**2-2*rho3*np.dot(P3,u3))

r1/au,r3/au

(1.0025929094077, 1.022134440877157)

# VECTORES EN COMPONENTES RECTANGULARES

In [ ]:
#Vectores de posición
r1vec=(rho1*u1-P1)
r2vec=(rho2*u2-P2)
r3vec=(rho3*u3-P3)

r1vec/au,r2vec/au,r3vec/au

(array([ 0.40143418, -0.8439039 , -0.3631382 ]),
 array([ 0.82380025, -0.53639389, -0.22974542]),
 array([ 1.01842274, -0.08059518, -0.03283841]))

In [ ]:
#Vector de velocidad
v2vec=(-f3*r1vec+f1*r3vec)/(f1*g3-f3*g1)

v2vec

array([18.06272316, 22.34633683,  9.6697315 ])

# CAMBIO AL SISTEMA ECLIPTICO

In [ ]:
e=23+26/60+12/3600 #ÁNGULO DE OBLICUIDAD  DE LA ECLÍPTICA
def vectores_sol (ra):
  sx=ra[0]
  sy= ra[1]*np.cos(e*deg)+ra[2]*np.sin(e*deg)
  sz=ra[2]*np.cos(e*deg)-ra[1]*np.sin(e*deg)
  
  return (sx,sy,sz)

In [ ]:
r1=np.array(vectores_sol(r1vec))
r2=np.array(vectores_sol(r2vec))
r3=np.array(vectores_sol(r3vec))

r1/au,r2/au,r3/au

(array([ 0.40143418, -0.91871488,  0.00247083]),
 array([ 0.82380025, -0.58351937,  0.0025512 ]),
 array([ 1.01842274, -0.08700709,  0.00192628]))

In [ ]:
e=23+26/60+12/3600 #ÁNGULO DE OBLICUIDAD  DE LA ECLÍPTICA
v2=np.array([v2vec[0],v2vec[1]*np.cos(e*deg)+v2vec[2]*np.sin(e*deg),v2vec[2]*np.cos(e*deg)-v2vec[1]*np.sin(e*deg)])
v2

array([ 1.80627232e+01,  2.43487622e+01, -1.59418128e-02])

# VECTORES DE ELEMENTOS ORBITALES

In [ ]:
#MOMENTO ANGULAR
h=np.cross(r2,v2)

#VECTOR DE NODO
k=np.array([0,0,1])
N=np.cross(k,h)

#VECTOR EXCENTRICIDAD
l=np.cross(v2,h)-mu*(r2/np.linalg.norm(r2))
e=l/mu

h,N,l,e

(array([-7.90119406e+06,  8.85836301e+06,  4.57746525e+09]),
 array([-8858363.01335058, -7901194.0561575 ,        0.        ]),
 array([ 3.15913985e+09, -5.97202356e+09,  1.70101410e+07]),
 array([ 0.0238044 , -0.04499973,  0.00012817]))

# ELEMENTOS ORBITALES

In [ ]:
#Inclinación
i=np.arccos(h[2]/np.linalg.norm(h))*rad

#Excentricidad
ex=np.linalg.norm(e)
if ex>1:
  ex=ex-1

#Longitud del nodo
if N[1]>=0:
  long_node = np.arccos(N[0]/np.linalg.norm(N))*rad 
  if long_node<0:
    long_node = long_node + 360
else:
  long_node = 2 * np.pi-np.arccos(N[0]/np.linalg.norm(N))*rad
  if long_node<0:
    long_node = long_node + 360

#Argumento del perihelio
if e[2]>=0:
  arg_perh = np.arccos(np.dot(N,e)/(np.linalg.norm(N)*np.linalg.norm(e)))*rad
  if arg_perh<0:
    arg_perh = arg_perh + 360
else:
  arg_perh = 2 * np.pi-np.arccos(np.dot(N,e)/(np.linalg.norm(N)*np.linalg.norm(e)))*rad
  if arg_perh<0:
    arg_perh = arg_perh + 360

#Anomalia verdadera,
A=np.dot(e,r2)
B=ex*np.linalg.norm(r2)
tetha=np.arccos(np.linalg.norm(A/B))*rad

#Semi-eje mayor
#semi-latus rectum
q=np.linalg.norm(h**2/mu)/au
a=q/(1-ex**2)


print('i=',i, 'a=',a ,'ex=', ex, 'longitud_nodo=', long_node )
print('argumento_perihelio=',arg_perh , 'tetha=', tetha )



i= 0.1485768927011881 a= 1.0581325220863862 ex= 0.050908164882779654 longitud_nodo= 228.01446938359294
argumento_perihelio= 76.14714663286213 tetha= 26.81058283875235


# GRÁFICA

In [ ]:
!pip install https://github.com/JohaMR/poliastro/archive/refs/heads/patch-1.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 5.2 MB 488 kB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for poliastro: filename=poliastro-0.16.dev0-py3-none-any.whl size=139010 sha256=8e83c24713a82eb4d13f98e6595b56c7b2dbfe8e796a6d3d8e21c12f3e3361e9
  Stored in directory: /root/.cache/pip/wheels/04/25/0f/b6c1e6efd2b14589cc0456f747ec3a194d0c22cd91fd812075
Successfully built poliastro
  Attempting uninstall: poliastro
    Found existing installation: poliastro 0.16.0
    Uninstalling poliastro-0.16.0:
      Successfully uninstalled poliastro-0.16.0


In [ ]:
epoca = Time("2021-05-30 14:10:00", scale="tdb")
epoca1 = Time("2021-07-01 00:00:00", scale="tdb")
Tierra=Orbit.from_classical(attractor= Sun,epoch=epoca, a=1.0*u.au, ecc=0.01673*u.one, inc=0*u.deg, raan=-11.26064*u.deg, argp=102.94719*u.deg, nu=100.46435*u.deg)
Marte=Orbit.from_classical(attractor= Sun, epoch=epoca, a=1.52366*u.au, ecc=0.09341*u.one, inc=1.85061*u.deg, raan=49.57854 *u.deg, argp=336.04084  *u.deg, nu=355.45332*u.deg)
Asteroid=Orbit.from_classical(attractor= Sun, epoch=epoca, a=a*u.au, ecc=ex*u.one, inc=i*u.deg, raan=long_node*u.deg, argp=arg_perh*u.deg, nu=20*u.deg)
#Asteroid1=Orbit.from_classical(attractor= Sun, epoch=epoca1, a=1.97054*u.au, ecc=0.083978*u.one, inc=22.376133*u.deg, raan=67.7242039*u.deg, argp=108.794248*u.deg, nu=68.5141522*u.deg)
frame = OrbitPlotter3D(unit=u.au) 
fig=frame.plot(Tierra, label="Tierra")
fig=frame.plot(Marte, label="Marte")
fig=frame.plot(Asteroid,label="Asteroide")
grafica=fig.show()

/usr/local/lib/python3.7/dist-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)
